In [33]:
import pandas as pd
import numpy as np
import sys
np.set_printoptions(precision=3)

In [34]:
trainData = pd.read_csv("1644871288_9762487_cleveland-train.csv")

In [35]:
print(trainData)

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     67    1   4       120   229    0        2      129      1      2.6   
1     37    1   3       130   250    0        0      187      0      3.5   
2     56    1   2       120   236    0        0      178      0      0.8   
3     57    0   4       120   354    0        0      163      1      0.6   
4     63    1   4       130   254    0        2      147      0      1.4   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
147   41    1   2       120   157    0        0      182      0      0.0   
148   59    1   4       164   176    1        2       90      0      1.0   
149   68    1   4       144   193    1        0      141      0      3.4   
150   57    1   4       130   131    0        0      115      1      1.2   
151   57    0   2       130   236    0        2      174      0      0.0   

     slope  ca  thal  heartdisease::category|-1|1  
0        2   2     7               

In [36]:
trainData.columns.values

array(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
       'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal',
       'heartdisease::category|-1|1'], dtype=object)

In [37]:
Y = trainData['heartdisease::category|-1|1']

In [38]:
X_true = trainData.loc[:, trainData.columns != 'heartdisease::category|-1|1'].to_numpy()

In [39]:
X = np.insert(X_true,0,1, axis = 1)

In [40]:
print(X.shape[1])

14


In [41]:
w = np.zeros(X.shape[1])
size = X.shape[0]


In [42]:
def CrossEntropyError(X, Y, w, size):
    B = np.dot(X, w[:, None]).reshape(-1)
    B = np.einsum ("i, i->i", -Y, B)
    B = np.exp(B) + 1
    error = np.sum( np.log(B), axis = 0) / size
    return error

In [43]:
def GradientDescent(X, Y, w, size, A):
    B = np.dot(X, w[:, None]).reshape(-1)
    B = np.einsum ("i, i->i", Y, B)
    B = np.exp(B) + 1
    C = np.tile(B.transpose(), (14, 1)).transpose()
    gt = (-1/ size) * np.sum( np.divide(A, C), axis = 0)
    return gt

In [44]:
def ModelTraining(maxIter, n, X, Y, w, size):
    A = np.einsum("i,ij->ij", Y,X)
    for i in range(1,maxIter):
        gt = GradientDescent(X, Y, w, size, A)
        a = np.max(gt)
        b = np.min(gt)
        if a < 0.00001 and b >-0.00001:
            print(i)
            break
        w = w - n*gt
    return w

In [45]:
%%time 
w = ModelTraining(10000000, 0.001, X, Y, w, size)


CPU times: user 3min 10s, sys: 543 ms, total: 3min 10s
Wall time: 3min 10s


In [14]:
def printFormat(X, Y, w, size):
    print( "Cross Entropy Error is " + str(CrossEntropyError(X, Y, w, size)))
    print( "Weight vector is " + str(w))
    print("Training Accuracy is " + str(predict(X,Y)) )

In [15]:
def predict(X, Y):
    
    B = 1/ (1 + np.exp( -1 * np.dot(X, w[:, None]).reshape(-1)))
    B[B >= 0.5] = 1
    B[B < 0.5] = -1
    return np.count_nonzero(np.equal(B,Y))/size

In [16]:
printFormat(X, Y, w, size)

Cross Entropy Error is 5.330597898899743
Weight vector is [-0.165 -0.254  1.281  2.647  0.138 -0.054 -1.226  1.749 -0.288  1.576
  2.645  0.647  5.5    2.553]
Training Accuracy is 0.6776315789473685


In [17]:
import sklearn
from sklearn.linear_model import LogisticRegression


In [18]:

model = LogisticRegression(max_iter = 10000)


In [19]:
%%time
predicted = model.fit(X,Y)

CPU times: user 74.9 ms, sys: 2.83 ms, total: 77.7 ms
Wall time: 75.9 ms


In [20]:
P = predicted.predict(X)
print("Training Accuracy of Sklearn's implementation is " + str(np.count_nonzero(np.equal(P,Y))/size))

Training Accuracy of Sklearn's implementation is 0.8223684210526315


In [21]:
mean = np.mean(X_true, axis = 0)
SD = np.std(X_true, axis = 0)
X_scaled = np.divide((X_true-mean), SD)

In [22]:
X_scaled = np.insert(X_scaled,0,1, axis = 1)
w1 = np.zeros(X_scaled.shape[1])

In [23]:
print(X_scaled)

[[ 1.     1.398  0.721 ...  0.7    1.49   1.133]
 [ 1.    -1.981  0.721 ...  2.361 -0.72  -0.929]
 [ 1.     0.159  0.721 ... -0.962 -0.72  -0.929]
 ...
 [ 1.     1.511  0.721 ...  0.7    1.49   1.133]
 [ 1.     0.272  0.721 ...  0.7    0.385  1.133]
 [ 1.     0.272 -1.387 ...  0.7    0.385 -0.929]]


In [24]:
%%time
w1 = ModelTraining(10000, 0.0001, X_scaled, Y, w1, size)

CPU times: user 215 ms, sys: 5.68 ms, total: 220 ms
Wall time: 217 ms


In [25]:
print(w)

[-0.165 -0.254  1.281  2.647  0.138 -0.054 -1.226  1.749 -0.288  1.576
  2.645  0.647  5.5    2.553]


In [26]:
printFormat(X_scaled, Y, w1, size)

Cross Entropy Error is 0.5236887625783183
Weight vector is [-0.058  0.067  0.109  0.142  0.052 -0.015 -0.006  0.053 -0.12   0.118
  0.132  0.094  0.168  0.186]
Training Accuracy is 0.8223684210526315


In [27]:
Predicted = model.fit(X_scaled, Y)

In [28]:

P = predicted.predict(X_scaled)
print("Training Accuracy of Sklearn's implementation for scaled data is " + str(np.count_nonzero(np.equal(P,Y))/size))

Training Accuracy of Sklearn's implementation for scaled data is 0.8157894736842105


In [29]:
print(w)

[-0.165 -0.254  1.281  2.647  0.138 -0.054 -1.226  1.749 -0.288  1.576
  2.645  0.647  5.5    2.553]


In [30]:
testData = pd.read_csv("1644871288_9775174_cleveland-test.csv")

In [31]:
Test = testData.to_numpy()

In [32]:
T = 1/ (1 + np.exp( -1 * np.dot(Test, w[1:, None]).reshape(-1)))
T[T >= 0.5] = 1
T[T < 0.5] = -1
np.savetxt("out.csv", T, delimiter=",", fmt='%d')